In [3]:
import pypyodbc

server = '192.168.0.21'
port = '4900'
database = 'BAC_A_SABLE'
username = 'admin'
password = 'Clip_SERENA'


cnxn = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

crsr = cnxn.cursor()

table_name = 'CHARGE'
#cf_name = 'C6800'
semaine = '12'

#query = 'SELECT * FROM {}'.format(table_name)
#query = 'SELECT * FROM ' +table_name+ ' WHERE NAF < 1000'
query = "SELECT * FROM " +table_name+ " WHERE COFRAIS like 'C35-2'"
#COFRAIS like '" +cf_name+ "' AND

records = crsr.execute(query).fetchall()
print(records)

#columns_list = list(crsr.columns())
#columns_list

[(86899, datetime.date(2024, 3, 20), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.217488, 0.0, 'SNCFR', 40792, 431668), (83571, datetime.date(2024, 3, 25), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.116875, 0.0, 'SNCFR', 40836, 432156), (114803, datetime.date(2024, 5, 2), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.796947, 0.0, 'SNCFM', 40871, 433073), (113779, datetime.date(2024, 4, 1), None, 'C35-2', 'ATCRM', 'C35-2                         ', 2.587678, 0.0, 'EWELL', 40211, 427836), (112755, datetime.date(2024, 4, 24), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.103599, 0.0, 'SNCFM', 40876, 433567), (86898, datetime.date(2024, 3, 21), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.217488, 0.0, 'SNCFR', 40792, 431668), (83570, datetime.date(2024, 3, 26), None, 'C35-2', 'ATCRM', 'C35-2                         ', 0.116875, 0.0, 'SNCFR', 40836, 432156), (114802, datetime.date(2024, 5, 3), None, 'C35-2', 'ATCRM', 

#### one_row = crsr.fetchone()
type(one_row)